# 初始化

In [ ]:
import os
from local_packages import *                                                                                                                                                                                                                                                                                                                                               
from Packages.Multi_Process.multi_process import MultiProcessor
from Packages.Formatter.data_processor import DataProcessor

# 初始化API
loader=LLMLoader(service_type='qwen', version='long')

llm=loader.service

# 初始化向量嵌入
embedder=QwenEmbedder()

# 初始化文字分割器
divider=TextDivider(threshold=512,overlap=0)
parser=LLMParser()
formatter=DataProcessor()

from Applications.GraphRAG.Config.entity_classifier import data_template, prompt, correction, validation
# 01
entity_classifier=MultiProcessor(llm,parser, data_template, prompt, correction, validation, isList=True)

from Applications.GraphRAG.Config.entity_extractor import data_template, prompt, correction, validation
# 02
entity_extractor=MultiProcessor(llm,parser, data_template, prompt, correction, validation, isList=True)

from Applications.GraphRAG.Config.tag_mount import data_template, prompt, correction, validation
# 03
tag_mount=MultiProcessor(llm,parser, data_template, prompt, correction, validation, isList=True)

from Applications.GraphRAG.Config.entity_explainer import data_template, prompt, correction, validation
# 04
entity_explainer=MultiProcessor(llm,parser, data_template, prompt, correction, validation, isList=True)

from Applications.GraphRAG.Config.relation_extractor import data_template, prompt, correction, validation

# 05
relation_extractor=MultiProcessor(llm,parser, data_template, prompt, correction, validation, isList=True)

# 主工作流的第一部分，得到近义词合并后的实体和类型

In [ ]:
input_chunk_list=divider.divide(txt_path='DataFolder/input.txt')

input_chunk_tuple_list=formatter.string2tuple_list(input_chunk_list)

type_tuple_list=entity_classifier.multitask_perform(input_chunk_tuple_list, num_threads=40)
print('step1 done')
entity_tuple_list=entity_extractor.multitask_perform(input_chunk_tuple_list, num_threads=40)
print('step2 done')
merged_type_tuple_list=formatter.convertor(embedder, type_tuple_list)

merged_entity_tuple_list=formatter.convertor(embedder,entity_tuple_list)

explanation_data=formatter.organize_data(input_chunk_tuple_list, merged_entity_tuple_list)

entity_explanation=entity_explainer.multitask_perform(explanation_data, num_threads=40)
print('step3 done')
relation_explanation=relation_extractor.multitask_perform(explanation_data, num_threads=40)
print('step4 done')
converted_entity_explanation = formatter.convert_structure(entity_explanation)

parser.write_json(relation_explanation, 'DataFolder/relation_explanation.json')
parser.write_json(converted_entity_explanation, 'DataFolder/entity_explanation.json')